In [13]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.document_loaders import PyPDFLoader
import pandas as pd
from PIL import Image
import base64
from langchain_core.messages import HumanMessage
import mimetypes
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
import dotenv
import os
dotenv.load_dotenv()


True

In [7]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash",temprature = 0)


Unexpected argument 'temprature' provided to ChatGoogleGenerativeAI. Did you mean: 'temperature'?
c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\interactiveshell.py:3577: UserWarning: WARNING! temprature is not default parameter.
                temprature was transferred to model_kwargs.
                Please confirm that temprature is what you intended.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
class UniversalLoader:
    def __init__(self,llm):
        self.llm = llm

    def process_file(self, file_path: str):
        """
        Traffic Controller: Routes files to the correct reader.
        """
        # 1. Get extension and mime type
        _, ext = os.path.splitext(file_path)
        ext = ext.lower()
        mime_type, _ = mimetypes.guess_type(file_path)
        
        # 2. DEFINE CODE EXTENSIONS (Treat these as text)
        code_extensions = {'.py', '.js', '.ts', '.html', '.css', '.java', '.cpp', '.c', '.h', '.sql', '.md', '.json', '.xml', '.yaml', '.yml', '.txt'}

        if ext in code_extensions:
            return self._process_code(file_path, ext)
        
        elif mime_type and "pdf" in mime_type:
            return self._process_pdf(file_path)
        
        elif mime_type and "csv" in mime_type:
            return self._process_csv(file_path)
        
        elif mime_type and "image" in mime_type:
            return self._process_image(file_path)
        
        else:
            return f"Unsupported file type: {mime_type or ext}"
        
    def _process_code(self, file_path, ext):
        """Reads code files and wraps them in markdown."""
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                content = f.read()
            # Wrap in markdown so LLM knows it's code
            lang_map = {'.py': 'python', '.js': 'javascript', '.ts': 'typescript', '.html': 'html', '.sql': 'sql', '.css': 'css'}
            language = lang_map.get(ext, '')
            return f"```{language}\n{content}\n```"
        except Exception as e:
            return f"Error reading code file: {e}"
        
    def _process_txt(self,file_path):
        with open(file_path,'r') as f:
            return f.read()
        
    def _process_pdf(self,file_path):
        loader = PyPDFLoader(file_path)
        pages = loader.load_and_split()
        result = ""
        for page in pages:
            result += page.page_content + "\n"
        return result
    
    def _process_csv(self,file_path):
        df = pd.read_csv(file_path)
        return df.to_markdown(index=False)
    
    def _process_image(self,file_path):
        try:
            with open(file_path, 'rb') as f:
                    
                image_data = base64.b64encode(f.read()).decode('utf-8')
                
                Prompt = HumanMessage(content = [
                    {"type":"text","text":"Describe the following image in detail."},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_data}"}},
                ])
                response = self.llm.invoke([Prompt])
                return response.content

        except Exception as e:
            return "error processing image: " + str(e)
        

universalloader = UniversalLoader(model)
    
    



In [9]:
search_tool = DuckDuckGoSearchRun()
search_tool.name = "search_tool"
search_tool.description = "web search tool to find more information about a topic"
result = search_tool.invoke("where is udaipur?")
print(result)

Restaurants and Local Food in Udaipur . Udaipur is where much of Rajasthan's grandeur resides, traces of which can be seen in the rich cuisine available here. Top 5 Reasons to Visit Udaipur . 1. The Stunning Lakeside Location. Udaipur is also known as the "City of Lakes", and it's easy to see why. MapsofIndia.com - Map showing the location of Udaipur ,Rajasthan in India. Find where is Udaipur located. Udaipur is located in Rajasthan, the desert state of India, roughly mid-way between New Delhi and Mumbai. Romantic, relaxed Udaipur is the perfect place. Here are our favourite things to do in Udaipur , plus advice on where to stay, where to eat, and how to get there.


In [10]:
tools = [search_tool]
model = model.bind_tools(tools)
#embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
vector_store = FAISS.from_texts(["Nexus Initialized"], embeddings)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_20444\1065513397.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [11]:
def index_files(file_paths):
    """
    Reads files -> Chunks them -> Saves to Vector DB (Locally)
    """
    all_documents = []
    
    for path in file_paths:
        print(f"Loading: {path}...")
        
        # Extract Text using your UniversalLoader
        raw_content = universalloader.process_file(path)
        
        # Convert to Document
        doc = Document(page_content=raw_content, metadata={"source": path})
        all_documents.append(doc)
        
    # Split into chunks
    splits = text_splitter.split_documents(all_documents)
    
    # Add to Vector Store
    if splits:
        vector_store.add_documents(splits)
        print(f"Successfully indexed {len(splits)} chunks locally!")
    else:
        print("No content found to index.")

In [14]:
with open("nexus_local_test.py", "w") as f:
    f.write("""
def secure_handshake():
    '''Local encryption protocol for Nexus'''
    return "Handshake Accepted"
""")

# Run the Indexer
index_files(["nexus_local_test.py"])

Loading: nexus_local_test.py...
Successfully indexed 1 chunks locally!


In [ ]:
retriever = vector_store.as_retriever(search_kwargs={"k": 2})